In [31]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [32]:
#Initial table set

#Moon equation form the Parisian Alfonsines Tables
Equation_center=HTable.read(181,format='dishas')
Equation_anomaly=HTable.read(182,format='dishas')
Div_diam=HTable.read(240,format='dishas')
Min_prop=HTable.read(239,format='dishas')


In [33]:
# Initial values

# Intial arguments of the table: moon argument at syzygy and number of days since last syzygy
REVO=Sexagesimal("6,0;0,0")*u.degree
moon_argument= Sexagesimal("6,0;0,0")*u.degree
moon_age=7

# moon mean motion parameter from the Parisian Alfonsine tables
daily_mean_center=Sexagesimal("13;10,35,1,15,11,4,35")*u.degree
daily_mean_arg=Sexagesimal("13;3,53,57,30,21,4,13")*u.degree
daily_mean_elong=Sexagesimal("12;11,26,41,37,51,50,39")*u.degree


(moon_argument, moon_age, daily_mean_center, daily_mean_elong, daily_mean_arg)

(<BasedQuantity 06,00 ; 00,00 deg>,
 7,
 <BasedQuantity 13 ; 10,35,01,15,11,04,35 deg>,
 <BasedQuantity 12 ; 11,26,41,37,51,50,39 deg>,
 <BasedQuantity 13 ; 03,53,57,30,21,04,13 deg>)

In [34]:
# computation of the initial mean center and mean argument

with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
    mean_elong=2*moon_age*daily_mean_elong
    mean_argument=moon_argument+daily_mean_arg*moon_age
    mean_moon_corr=moon_age*daily_mean_center

(mean_elong, mean_argument, mean_moon_corr)

(<BasedQuantity 03,48 ; 45,05 deg>,
 <BasedQuantity 03,11 ; 02,01 deg>,
 <BasedQuantity 04,36 ; 48,35 deg>)

In [35]:
# computation of the equation of center and min prop

with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
    eq_center=Equation_center.get(mean_elong%REVO)
    min_prop1=Min_prop.get(mean_elong%REVO)

(eq_center, min_prop1)

(<BasedQuantity -12 ; 21,15,35 deg>, <BasedQuantity 47 ; 14,55,00 deg>)

In [36]:
# Computation of the true argument

with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
    true_argument=mean_argument+eq_center   
true_argument

<BasedQuantity 02,58 ; 40,45,25 deg>

In [37]:
# computation of equation of anomaly

with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
    eq_anomaly1=Equation_anomaly.get(true_argument%REVO)
    div_diam1=Div_diam.get(true_argument%REVO)
    min_prop2=min_prop1*div_diam1/(60*u.degree)
    
#canons instruct to always add min_prop2 but this is not what JoL is doing
# he seems to apply to min prop the rule for the global equation  

    if eq_anomaly1>0:
        eq_anomaly2=eq_anomaly1+min_prop2
    else:
        eq_anomaly2=eq_anomaly1-min_prop2
    
(eq_anomaly1, div_diam1, min_prop2, eq_anomaly2)

(<BasedQuantity -00 ; 07,28,25 deg>,
 <BasedQuantity 00 ; 05,16,59 deg>,
 <BasedQuantity 0 ; 04,09,37 deg>,
 <BasedQuantity -00 ; 11,38,02 deg>)

In [38]:
# computation of the final correction and transfomation of the results to historical format

with set_precision(tmode=TruncatureMode.ROUND, pmode=1):
    moon_corr_fin=mean_moon_corr+eq_anomaly2
    moon_argument=Historical(moon_argument.value,1)
    moon_corr_fin=Historical(moon_corr_fin.value,1)

('moon_argument', moon_argument, 'moon_age', moon_age, 'moon_corr_fin', moon_corr_fin)

('moon_argument', 1r 00s 00 ; 00, 'moon_age', 7, 'moon_corr_fin', 09s 06 ; 37)